# Nettoyage des donnees avant de les enregistrer en BDD

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import sqlite3
import os

from modules.data_cleaning import apply_zscore

raw = '../../datas/RAW/E2/'

# Chargement des csv

In [2]:
dict_csv = {
    'customer':         'olist_customers_dataset.csv', 
    'geolocalisation':  'olist_geolocation_dataset.csv', 
    'order':            'olist_orders_dataset.csv', 
    'order_items':      'olist_order_items_dataset.csv', 
    'product':          'olist_products_dataset.csv', 
    'seller':           'olist_sellers_dataset.csv'
}
for csv in dict_csv:
    dict_csv[csv] = (pd.read_csv(raw + dict_csv[csv]))
dict_csv.keys()

dict_keys(['customer', 'geolocalisation', 'order', 'order_items', 'product', 'seller'])

# Nettoyage des donnees

## Drop na values

sur la table order, j'en profite pour supprimer les 6 commandes annulees (canceled)

In [3]:
for df in dict_csv:
    if dict_csv[df].isna().sum().sum() != 0:
        print('\t' + df.upper())
        print(dict_csv[df].isna().sum())

	ORDER
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64
	PRODUCT
product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64


In [4]:
order_without_na = dict_csv['order'].dropna()
order_without_na = order_without_na[order_without_na.order_status == 'delivered']
print('Nb de lignes avant et apres traitement')
print(dict_csv['order'].shape)
print(order_without_na.shape)
# order_without_na.isna().any()

(99441, 8)
(96455, 8)


order_id                         False
customer_id                      False
order_status                     False
order_purchase_timestamp         False
order_approved_at                False
order_delivered_carrier_date     False
order_delivered_customer_date    False
order_estimated_delivery_date    False
dtype: bool

In [5]:
product_without_na = dict_csv['product'].dropna()
print('Nb de lignes avant et apres traitement')
print(dict_csv['product'].shape)
print(product_without_na.shape)
# product_without_na.isna().any()

(32951, 9)
(32340, 9)


product_id                    False
product_category_name         False
product_name_lenght           False
product_description_lenght    False
product_photos_qty            False
product_weight_g              False
product_length_cm             False
product_height_cm             False
product_width_cm              False
dtype: bool

## Drop outliers

J'applique un zscore uniquement sur la table order_items

In [ ]:
order_items_zscore = apply_zscore(order_items, ['price', 'freight_value'])
ax = sns.boxplot(data=order_items_zscore)
order_items_zscore.shape

# Enregistrement en base de donnees

In [8]:
clean = '../../datas/CLEAN/E2/'

if os.path.exists(clean) == False:
    os.makedirs(clean)
con = sqlite3.connect(clean + 'E2.db')

In [9]:
con.close()